In [2]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [3]:
con = psycopg2.connect(host = 'localhost', user = 'postgres', password = '0110casa')
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = con.cursor()
nome_banco = 'cadastro'
sql_criar_banco = 'CREATE DATABASE {}'.format(nome_banco) 
try:
    cursor.execute(sql_criar_banco)
    print('Banco {} criado com sucesso !'.format(nome_banco))
except:
    print('Banco {} já existe !'.format(nome_banco))

Banco cadastro já existe !


In [35]:
con.close()

In [4]:
con = psycopg2.connect(host = 'localhost', database='cadastro' ,user = 'postgres', password = '0110casa')
cur = con.cursor()

In [5]:
query = "CREATE TABLE pessoas (id BIGSERIAL PRIMARY KEY, nome VARCHAR(100) NOT NULL, idade INT NOT NULL, endereco VARCHAR (100))";
cur.execute(query)
con.commit()

In [5]:
from flask import Flask, request

In [20]:
app = Flask(__name__)

@app.route('/nova_pessoa' , methods = ['POST'])
def nova_pessoa():
    post_args = request.json
    if 'nome' in post_args and 'idade' in post_args:
        query = "INSERT INTO pessoas (id, nome, idade, endereco) VALUES (default,'{nome}', '{idade}', '{endereco}');".format(nome=post_args ['nome'], idade=post_args ['idade'], endereco=post_args['endereco'])
        cur.execute(query)
        con.commit()
        return "Pessoa salva com sucesso ! ", 200
    return "Dado recebido está errado ou incompleto. ", 400
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2020 20:40:22] "POST /nova_pessoa HTTP/1.1" 200 -


In [33]:
import pandas as pd
app = Flask(__name__)
@app.route("/selecionar_pessoas", methods=['GET'])
def selecionar_pessoas():
    query = "SELECT * FROM pessoas"
    try:
        df = pd.read_sql(query, con)
        print(df.head(10))
        return df.to_json(orient='records'), 200
    except:
        return "Não foi possível fazer SELECT na tabela pessoas.", 500
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2020 20:46:42] "GET /selecionar_pessoas HTTP/1.1" 200 -


   id     nome  idade                endereco
0   1  William     34  Rua Rafael Ficondo 747


In [7]:
import pandas as pd
app = Flask(__name__)
@app.route("/selecionar_pessoa/<id>", methods=['GET'])
def selecionar_pessoa(id):
    query = "SELECT * FROM pessoas WHERE id = {id}".format(id=id)
    try:
        df = pd.read_sql(query, con)
        print(df)
        return df.to_json(orient='records'), 200
    except:
        return "Não foi possível fazer SELECT na tabela pessoas.", 500
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2020 20:51:54] "GET /selecionar_pessoa/%3C1%3E HTTP/1.1" 500 -
127.0.0.1 - - [07/Feb/2020 20:52:20] "GET /selecionar_pessoa/1 HTTP/1.1" 200 -


   id     nome  idade                endereco
0   1  William     34  Rua Rafael Ficondo 747


127.0.0.1 - - [07/Feb/2020 20:54:09] "PUT /atualizar_pessoa HTTP/1.1" 404 -


In [9]:
app = Flask(__name__)
@app.route("/atualizar_pessoa", methods=['PUT'])
def atualizar_pessoa():
    post_args = request.json
    if "id" not in post_args:
        return "O id não foi informado no dado recebido.", 400
    
    if "nome" in post_args:
        query = "UPDATE pessoas SET nome = '{nome}' WHERE id = '{id}'".format(nome=post_args["nome"], id=post_args["id"])
        cur.execute(query)
    if "idade" in post_args:
        query = "UPDATE pessoas SET idade = '{idade}' WHERE id = '{id}'".format(idade=post_args["idade"], id=post_args["id"])
        cur.execute(query)
    if "endereco" in post_args:
        query = "UPDATE pessoas SET endereco = '{endereco}' WHERE id = '{id}'".format(endereco=post_args["endereco"], id=post_args["id"])
        cur.execute(query)
    con.commit()
    return "Pessoa atualizada com sucesso!", 200

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2020 20:54:35] "PUT /atualizar_pessoa HTTP/1.1" 400 -
127.0.0.1 - - [07/Feb/2020 20:55:28] "PUT /atualizar_pessoa HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2020 20:56:41] "PUT /atualizar_pessoa HTTP/1.1" 200 -


In [ ]:
app = Flask(__name__)
@app.route("/deletar_pessoa", methods=['DELETE'])
def deletar_pessoa():
    post_args = request.json
    if "id" not in post_args:
        return "O id não foi informado no dado recebido.", 400
    
    query = "DELETE FROM pessoas WHERE id = {id};".format(id=post_args["id"])
    cur.execute(query)
    con.commit()
    return "Pessoa deletada com sucesso!", 200

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2020 21:00:36] "DELETE /deletar_pessoa HTTP/1.1" 200 -
